In [ ]:

import os
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql.functions import *

#Присваеваем переменные исходным таблицам из staging

countries_of_the_world=spark.sql("select * from andreynovikov_staging.countries_of_the_world_orc")
capital=spark.sql("select * from andreynovikov_staging.capital")
continent=spark.sql("select * from andreynovikov_staging.continent")
currency=spark.sql("select * from andreynovikov_staging.currency")
iso3=spark.sql("select * from andreynovikov_staging.iso3")
names=spark.sql("select * from andreynovikov_staging.names")
phone=spark.sql("select * andreynovikov_staging.phone")
nobel_laureates=spark.sql("select * andreynovikov_staging.nobel_laureates_orc")
cities=spark.sql("select * from andreynovikov_staging.cities_orc")

#Соединяем данные из json-файлов в таблицу names_mod

joinExp_cap = names["country_code"] == capital["country_code"]
joinExp_con = names["country_code"] == continent["country_code"]
joinExp_cur = names["country_code"] == currency["country_code"]
joinExp_iso = names["country_code"] == iso3["country_code"]
joinExp_pho = names["country_code"] == phone["country_code"]
joinType = "outer"
names_mod=names.join(iso3, joinExp_iso, joinType).drop(iso3['country_code']) \
               .join(continent, joinExp_con, joinType).drop(continent['country_code']) \
               .join(capital, joinExp_cap, joinType).drop(capital['country_code']) \
               .join(currency, joinExp_cur, joinType).drop(currency['country_code']) \
               .join(phone, joinExp_pho, joinType).drop(phone['country_code'])
               
#Для соединения countries_of_the_world с names_mod убираем пробелы в значениях названий стран в countries_of_the_world и заменяем '&' на 'and'.

countries_of_the_world=countries_of_the_world.withColumn("country", rtrim(countries_of_the_world['country']))
countries_of_the_world=countries_of_the_world.withColumn("country", ltrim(countries_of_the_world['country']))
countries_of_the_world=countries_of_the_world.withColumn("country", regexp_replace(countries_of_the_world['country'],'&','and'))

In [ ]:
#Соединяем countries_of_the_world с данными из json-файлов
#В name_mod (данные из json-файлов) 250 строк, в countries_of_the_world 227 строк. Совпадений по названиям стран - 206. Чтобы не терять данные по несовпадающим строкам делаем таблицу, где будут все строки из обеих таблиц, но с одной колонкой с названием страны, и добавляем колонку с ID.

#Делаем таблицу со всеми строками из countries_of_the_world и одной колонкой с названием страны

countries_of_the_world1=countries_of_the_world.join(names_mod, countries_of_the_world["country"] == names_mod["country_name"], 'left_outer').drop(names_mod['country_name'])

#Делаем таблицу со всеми строками из всех таблиц, затем выбираем из них строки из names_mod, которые не нашли совпадений, переименовываем в них колонку с названием страны, присоединяем получившиеся строки к countries_of_the_world1, добавляем колонку с ID, записываем результат в базу данных для снежинки.

countries_of_the_world=countries_of_the_world.join(names_mod, countries_of_the_world["country"] == names_mod["country_name"], "outer") \
                                             .where(f.col('country').isNull()) \
                                             .drop(countries_of_the_world['country']) \
                                             .withColumnRenamed('country_name','country') \
                                             .unionByName(countries_of_the_world1) \
                                             .withColumn('ID', f.monotonically_increasing_id())

In [ ]:
#Преобразуем cities. Меняем название страны на ID из стран мира

#В cities не идентифицируется 23 тыс строк по странам с кодом стран ZR и AN. Для них оставим такое обозначение, для остальных заменим код страны на ID страны. Добавим собственный ID для городов.

cities=cities.withColumn("country", f.upper(cities.country))
cities_zr_an=cities.join(countries_of_the_world, cities["country"] == countries_of_the_world["country_code"], "left_anti") \
                   .withColumnRenamed('country', 'country_ID')

cities=cities.join(countries_of_the_world.select('ID', 'country_code'), cities["country"] == countries_of_the_world["country_code"], "inner") \
             .drop('country_code', 'country') \
             .withColumnRenamed('ID', 'country_ID') \
             .unionByName(cities_zr_an) \
             .withColumn('ID', f.monotonically_increasing_id())

In [ ]:
#Преобразуем лауретов. Разделяем на 4 таблицы: лауреаты, информация об оргаизации, о рождении и о смерти. В последние три добавляем ID соответствующих стран и городов.

# Делаем названия городов и стран в "лауретах" большими буквами
nobel_laureates=nobel_laureates.withColumn("organization_country", f.upper(nobel_laureates.organization_country)) \
                               .withColumn("organization_city", f.upper(nobel_laureates.organization_city)) \
                               .withColumn("birth_country", f.upper(nobel_laureates.birth_country)) \
                               .withColumn("birth_city", f.upper(nobel_laureates.birth_city)) \
                               .withColumn("death_country", f.upper(nobel_laureates.death_country)) \
                               .withColumn("death_city", f.upper(nobel_laureates.death_city))

# Соединяем города со странами
cities_countries=cities.withColumnRenamed('region','city_region') \
                       .withColumnRenamed('population','population_city') \
                       .withColumnRenamed('ID','city_id') \
                       .join(countries_of_the_world, cities['country_ID'] == countries_of_the_world["ID"], "left_outer") \
                       .drop('country_ID') \
                       .withColumnRenamed('ID','country_id')

# Делаем названия городов и стран в "города+страны" большими буквами
cities_countries=cities_countries.withColumn("country", f.upper(cities_countries.country)) \
                                 .withColumn("city", f.upper(cities_countries.city))

# Выбираем нужные колонки для лауреатов, id добавляем.
nobel_laureates=nobel_laureates.withColumn('unique_laureate_id', f.monotonically_increasing_id())
laureates=nobel_laureates.select('unique_laureate_id',
                                 'year',
                                 'category',
                                 'prize',
                                 'motivation',
                                 'prize_share',
                                 'laureate_id',
                                 'laureate_type',
                                 'full_name',
                                 'sex') 

# Выбираем нужные колонки для таблицы организации, добавляем свой id, отфильтровываем пустые строки, соединяем с ID из стран мира.
join_exp_org = (nobel_laureates["organization_country"] == cities_countries["country"]) & (nobel_laureates["organization_city"] == cities_countries["city"])
organization=nobel_laureates.select(f.monotonically_increasing_id().alias('organization_id'),
                                    'unique_laureate_id',
                                    'organization_name',
                                    'organization_city',
                                    'organization_country') \
                            .filter("organization_name > ''") \
                            .join(cities_countries.select('country_id', 'city_id', 'city', 'country'), join_exp_org, "left_outer") \
                            .drop('country', 'city') \
                            .withColumnRenamed('country_id', 'organization_country_id') \
                            .withColumnRenamed('city_id', 'organization_city_id')

# Выбираем нужные колонки для таблицы рождения, добавляем свой id, отфильтровываем пустые строки, соединяем с ID из стран мира.
join_exp_birth = (nobel_laureates["birth_country"] == cities_countries["country"]) & (nobel_laureates["birth_city"] == cities_countries["city"])
birth=nobel_laureates.select(f.monotonically_increasing_id().alias('birth_id'),
                                    'unique_laureate_id',
                                    'birth_date',
                                    'birth_city',
                                    'birth_country') \
                            .filter("birth_date > ''") \
                            .join(cities_countries.select('country_id', 'city_id', 'city', 'country'), join_exp_birth, "left_outer") \
                            .drop('country', 'city') \
                            .withColumnRenamed('country_id', 'birth_country_id') \
                            .withColumnRenamed('city_id', 'birth_city_id')

# Выбираем нужные колонки для таблицы смерти, добавляем свой id, отфильтровываем пустые строки, соединяем с ID из стран мира.
join_exp_death = (nobel_laureates["death_country"] == cities_countries["country"]) & (nobel_laureates["death_city"] == cities_countries["city"])
death=nobel_laureates.select(f.monotonically_increasing_id().alias('death_id'),
                                    'unique_laureate_id',
                                    'death_date',
                                    'death_city',
                                    'death_country') \
                            .filter("death_date > ''") \
                            .join(cities_countries.select('country_id', 'city_id', 'city', 'country'), join_exp_death, "left_outer") \
                            .drop('country', 'city') \
                            .withColumnRenamed('country_id', 'death_country_id') \
                            .withColumnRenamed('city_id', 'death_city_id')

In [ ]:
#Делаем Dataset

# Соединяем лауретов с городами отдельно для городов/стран рождения, смерти и организации
join_Expr_org = (cities_countries['city']==nobel_laureates['organization_city']) & (cities_countries['country']==nobel_laureates['organization_country'])
join_Expr_birth = (cities_countries['city']==nobel_laureates['birth_city']) & (cities_countries['country']==nobel_laureates['birth_country'])
join_Expr_death = (cities_countries['city']==nobel_laureates['death_city']) & (cities_countries['country']==nobel_laureates['death_country'])

dataset=nobel_laureates.join(cities_countries, join_Expr_org, 'left_outer') \
                       .withColumnRenamed('country','organization_city_country') \
                       .withColumnRenamed('accentcity','organization_city_accentcity') \
                       .withColumnRenamed('city_region','organization_city_region') \
                       .withColumnRenamed('population_city','organization_city_population') \
                       .withColumnRenamed('latitude','organization_city_latitude') \
                       .withColumnRenamed('longitude','organization_city_longitude') \
                       .withColumnRenamed('region','organization_country_region') \
                       .withColumnRenamed('population','organization_country_population') \
                       .withColumnRenamed('area','organization_country_area') \
                       .withColumnRenamed('pop_density','organization_country_pop_density') \
                       .withColumnRenamed('coastline','organization_country_coastline') \
                       .withColumnRenamed('net_migration','organization_country_net_migration') \
                       .withColumnRenamed('infant_mortality','organization_country_infant_mortality') \
                       .withColumnRenamed('gdp', 'organization_country_gdp') \
                       .withColumnRenamed('literacy','organization_country_literacy') \
                       .withColumnRenamed('phones','organization_country_phones') \
                       .withColumnRenamed('arable','organization_country_arable') \
                       .withColumnRenamed('crops','organization_country_crops') \
                       .withColumnRenamed('other','organization_country_other') \
                       .withColumnRenamed('climate','organization_country_climate') \
                       .withColumnRenamed('birthrate','organization_country_birthrate') \
                       .withColumnRenamed('deathrate','organization_country_deathrate') \
                       .withColumnRenamed('agriculture','organization_country_agriculture') \
                       .withColumnRenamed('industry','organization_country_industry') \
                       .withColumnRenamed('service','organization_country_service') \
                       .withColumnRenamed('country_code','organization_country_code') \
                       .withColumnRenamed('country','organization_country') \
                       .withColumnRenamed('country_iso3code','organization_country_iso3code') \
                       .withColumnRenamed('continent_code','organization_country_continent_code') \
                       .withColumnRenamed('capital','organization_country_capital') \
                       .withColumnRenamed('currency','organization_country_currency') \
                       .withColumnRenamed('phone','organization_country_phone') \
                       .drop('city') \
                       .join(cities_countries, join_Expr_birth, 'left_outer') \
                       .withColumnRenamed('country','birth_city_country') \
                       .withColumnRenamed('accentcity','birth_city_accentcity') \
                       .withColumnRenamed('city_region','birth_city_region') \
                       .withColumnRenamed('population_city','birth_city_population') \
                       .withColumnRenamed('latitude','birth_city_latitude') \
                       .withColumnRenamed('longitude','birth_city_longitude') \
                       .withColumnRenamed('region','birth_country_region') \
                       .withColumnRenamed('population','birth_country_population') \
                       .withColumnRenamed('area','birth_country_area') \
                       .withColumnRenamed('pop_density','birth_country_pop_density') \
                       .withColumnRenamed('coastline','birth_country_coastline') \
                       .withColumnRenamed('net_migration','birth_country_net_migration') \
                       .withColumnRenamed('infant_mortality','birth_country_infant_mortality') \
                       .withColumnRenamed('gdp', 'birth_country_gdp') \
                       .withColumnRenamed('literacy','birth_country_literacy') \
                       .withColumnRenamed('phones','birth_country_phones') \
                       .withColumnRenamed('arable','birth_country_arable') \
                       .withColumnRenamed('crops','birth_country_crops') \
                       .withColumnRenamed('other','birth_country_other') \
                       .withColumnRenamed('climate','birth_country_climate') \
                       .withColumnRenamed('birthrate','birth_country_birthrate') \
                       .withColumnRenamed('deathrate','birth_country_deathrate') \
                       .withColumnRenamed('agriculture','birth_country_agriculture') \
                       .withColumnRenamed('industry','birth_country_industry') \
                       .withColumnRenamed('service','birth_country_service') \
                       .withColumnRenamed('country_code','birth_country_code') \
                       .withColumnRenamed('country','birth_country') \
                       .withColumnRenamed('country_iso3code','birth_country_iso3code') \
                       .withColumnRenamed('continent_code','birth_country_continent_code') \
                       .withColumnRenamed('capital','birth_country_capital') \
                       .withColumnRenamed('currency','birth_country_currency') \
                       .withColumnRenamed('phone','birth_country_phone') \
                       .drop('city') \
                       .join(cities_countries, join_Expr_death, 'left_outer') \
                       .withColumnRenamed('country','death_city_country') \
                       .withColumnRenamed('accentcity','death_city_accentcity') \
                       .withColumnRenamed('city_region','death_city_region') \
                       .withColumnRenamed('population_city','death_city_population') \
                       .withColumnRenamed('latitude','death_city_latitude') \
                       .withColumnRenamed('longitude','death_city_longitude') \
                       .withColumnRenamed('region','death_country_region') \
                       .withColumnRenamed('population','death_country_population') \
                       .withColumnRenamed('area','death_country_area') \
                       .withColumnRenamed('pop_density','death_country_pop_density') \
                       .withColumnRenamed('coastline','death_country_coastline') \
                       .withColumnRenamed('net_migration','death_country_net_migration') \
                       .withColumnRenamed('infant_mortality','death_country_infant_mortality') \
                       .withColumnRenamed('gdp', 'death_country_gdp') \
                       .withColumnRenamed('literacy','death_country_literacy') \
                       .withColumnRenamed('phones','death_country_phones') \
                       .withColumnRenamed('arable','death_country_arable') \
                       .withColumnRenamed('crops','death_country_crops') \
                       .withColumnRenamed('other','death_country_other') \
                       .withColumnRenamed('climate','death_country_climate') \
                       .withColumnRenamed('birthrate','death_country_birthrate') \
                       .withColumnRenamed('deathrate','death_country_deathrate') \
                       .withColumnRenamed('agriculture','death_country_agriculture') \
                       .withColumnRenamed('industry','death_country_industry') \
                       .withColumnRenamed('service','death_country_service') \
                       .withColumnRenamed('country_code','death_country_code') \
                       .withColumnRenamed('country','death_country') \
                       .withColumnRenamed('country_iso3code','death_country_iso3code') \
                       .withColumnRenamed('continent_code','death_country_continent_code') \
                       .withColumnRenamed('capital','death_country_capital') \
                       .withColumnRenamed('currency','death_country_currency') \
                       .withColumnRenamed('phone','death_country_phone') \
                       .drop('city')

# Соединяем лауретов с городами отдельно для городов/стран рождения, смерти и организации
join_Expr_org = (cities_countries['city']==nobel_laureates['organization_city']) & (cities_countries['country']==nobel_laureates['organization_country'])
join_Expr_birth = (cities_countries['city']==nobel_laureates['birth_city']) & (cities_countries['country']==nobel_laureates['birth_country'])
join_Expr_death = (cities_countries['city']==nobel_laureates['death_city']) & (cities_countries['country']==nobel_laureates['death_country'])

dataset=nobel_laureates.join(cities_countries, join_Expr_org, 'left_outer') \
                       .withColumnRenamed('country','organization_city_country') \
                       .withColumnRenamed('accentcity','organization_city_accentcity') \
                       .withColumnRenamed('city_region','organization_city_region') \
                       .withColumnRenamed('population_city','organization_city_population') \
                       .withColumnRenamed('latitude','organization_city_latitude') \
                       .withColumnRenamed('longitude','organization_city_longitude') \
                       .withColumnRenamed('region','organization_country_region') \
                       .withColumnRenamed('population','organization_country_population') \
                       .withColumnRenamed('area','organization_country_area') \
                       .withColumnRenamed('pop_density','organization_country_pop_density') \
                       .withColumnRenamed('coastline','organization_country_coastline') \
                       .withColumnRenamed('net_migration','organization_country_net_migration') \
                       .withColumnRenamed('infant_mortality','organization_country_infant_mortality') \
                       .withColumnRenamed('gdp', 'organization_country_gdp') \
                       .withColumnRenamed('literacy','organization_country_literacy') \
                       .withColumnRenamed('phones','organization_country_phones') \
                       .withColumnRenamed('arable','organization_country_arable') \
                       .withColumnRenamed('crops','organization_country_crops') \
                       .withColumnRenamed('other','organization_country_other') \
                       .withColumnRenamed('climate','organization_country_climate') \
                       .withColumnRenamed('birthrate','organization_country_birthrate') \
                       .withColumnRenamed('deathrate','organization_country_deathrate') \
                       .withColumnRenamed('agriculture','organization_country_agriculture') \
                       .withColumnRenamed('industry','organization_country_industry') \
                       .withColumnRenamed('service','organization_country_service') \
                       .withColumnRenamed('country_code','organization_country_code') \
                       .withColumnRenamed('country','organization_country') \
                       .withColumnRenamed('country_iso3code','organization_country_iso3code') \
                       .withColumnRenamed('continent_code','organization_country_continent_code') \
                       .withColumnRenamed('capital','organization_country_capital') \
                       .withColumnRenamed('currency','organization_country_currency') \
                       .withColumnRenamed('phone','organization_country_phone') \
                       .drop('city') \
                       .join(cities_countries, join_Expr_birth, 'left_outer') \
                       .withColumnRenamed('country','birth_city_country') \
                       .withColumnRenamed('accentcity','birth_city_accentcity') \
                       .withColumnRenamed('city_region','birth_city_region') \
                       .withColumnRenamed('population_city','birth_city_population') \
                       .withColumnRenamed('latitude','birth_city_latitude') \
                       .withColumnRenamed('longitude','birth_city_longitude') \
                       .withColumnRenamed('region','birth_country_region') \
                       .withColumnRenamed('population','birth_country_population') \
                       .withColumnRenamed('area','birth_country_area') \
                       .withColumnRenamed('pop_density','birth_country_pop_density') \
                       .withColumnRenamed('coastline','birth_country_coastline') \
                       .withColumnRenamed('net_migration','birth_country_net_migration') \
                       .withColumnRenamed('infant_mortality','birth_country_infant_mortality') \
                       .withColumnRenamed('gdp', 'birth_country_gdp') \
                       .withColumnRenamed('literacy','birth_country_literacy') \
                       .withColumnRenamed('phones','birth_country_phones') \
                       .withColumnRenamed('arable','birth_country_arable') \
                       .withColumnRenamed('crops','birth_country_crops') \
                       .withColumnRenamed('other','birth_country_other') \
                       .withColumnRenamed('climate','birth_country_climate') \
                       .withColumnRenamed('birthrate','birth_country_birthrate') \
                       .withColumnRenamed('deathrate','birth_country_deathrate') \
                       .withColumnRenamed('agriculture','birth_country_agriculture') \
                       .withColumnRenamed('industry','birth_country_industry') \
                       .withColumnRenamed('service','birth_country_service') \
                       .withColumnRenamed('country_code','birth_country_code') \
                       .withColumnRenamed('country','birth_country') \
                       .withColumnRenamed('country_iso3code','birth_country_iso3code') \
                       .withColumnRenamed('continent_code','birth_country_continent_code') \
                       .withColumnRenamed('capital','birth_country_capital') \
                       .withColumnRenamed('currency','birth_country_currency') \
                       .withColumnRenamed('phone','birth_country_phone') \
                       .drop('city') \
                       .join(cities_countries, join_Expr_death, 'left_outer') \
                       .withColumnRenamed('country','death_city_country') \
                       .withColumnRenamed('accentcity','death_city_accentcity') \
                       .withColumnRenamed('city_region','death_city_region') \
                       .withColumnRenamed('population_city','death_city_population') \
                       .withColumnRenamed('latitude','death_city_latitude') \
                       .withColumnRenamed('longitude','death_city_longitude') \
                       .withColumnRenamed('region','death_country_region') \
                       .withColumnRenamed('population','death_country_population') \
                       .withColumnRenamed('area','death_country_area') \
                       .withColumnRenamed('pop_density','death_country_pop_density') \
                       .withColumnRenamed('coastline','death_country_coastline') \
                       .withColumnRenamed('net_migration','death_country_net_migration') \
                       .withColumnRenamed('infant_mortality','death_country_infant_mortality') \
                       .withColumnRenamed('gdp', 'death_country_gdp') \
                       .withColumnRenamed('literacy','death_country_literacy') \
                       .withColumnRenamed('phones','death_country_phones') \
                       .withColumnRenamed('arable','death_country_arable') \
                       .withColumnRenamed('crops','death_country_crops') \
                       .withColumnRenamed('other','death_country_other') \
                       .withColumnRenamed('climate','death_country_climate') \
                       .withColumnRenamed('birthrate','death_country_birthrate') \
                       .withColumnRenamed('deathrate','death_country_deathrate') \
                       .withColumnRenamed('agriculture','death_country_agriculture') \
                       .withColumnRenamed('industry','death_country_industry') \
                       .withColumnRenamed('service','death_country_service') \
                       .withColumnRenamed('country_code','death_country_code') \
                       .withColumnRenamed('country','death_country') \
                       .withColumnRenamed('country_iso3code','death_country_iso3code') \
                       .withColumnRenamed('continent_code','death_country_continent_code') \
                       .withColumnRenamed('capital','death_country_capital') \
                       .withColumnRenamed('currency','death_country_currency') \
                       .withColumnRenamed('phone','death_country_phone') \
                       .drop('city')

In [ ]:
# Записываем таблицы в Hive

countries_of_the_world.write.format("orc") \
                      .mode('overwrite') \
                      .option("compression","zlib") \
                      .saveAsTable("andreynovikov_snowflake.countries")

cities.write.format("orc") \
      .mode('overwrite') \
      .option("compression","zlib") \
      .saveAsTable("andreynovikov_snowflake.cities")

laureates.write.format("orc") \
         .mode('overwrite') \
         .option("compression","zlib") \
         .saveAsTable("andreynovikov_snowflake.laureates")

organization.write.format("orc") \
            .mode('overwrite') \
            .option("compression","zlib") \
            .saveAsTable("andreynovikov_snowflake.organization")

birth.write.format("orc") \
     .mode('overwrite') \
     .option("compression","zlib") \
     .saveAsTable("andreynovikov_snowflake.birth")

death.write.format("orc") \
     .mode('overwrite') \
     .option("compression","zlib") \
     .saveAsTable("andreynovikov_snowflake.death")

dataset.select('unique_laureate_id',
               'year',
               'category',
               'prize',
               'motivation',
               'prize_share',
               'laureate_id',
               'laureate_type',
               'full_name',
               'sex',
               
               'organization_name',
               'organization_city',
               'organization_city_accentcity',
               'organization_city_region',
               'organization_city_population',
               'organization_city_latitude',
               'organization_city_longitude',
               'organization_city_country',
               'organization_country_code',
               'organization_country_iso3code',
               'organization_country_continent_code',
               'organization_country_capital',
               'organization_country_currency',
               'organization_country_phone',
               'organization_country_region',
               'organization_country_population',
               'organization_country_area',
               'organization_country_pop_density',
               'organization_country_coastline',
               'organization_country_net_migration',
               'organization_country_infant_mortality',
               'organization_country_gdp',
               'organization_country_literacy',
               'organization_country_phones',
               'organization_country_arable',
               'organization_country_crops',
               'organization_country_other',
               'organization_country_climate',
               'organization_country_birthrate',
               'organization_country_deathrate',
               'organization_country_agriculture',
               'organization_country_industry',
               'organization_country_service',
               
               'birth_date',
               'birth_city',
               'birth_city_accentcity',
               'birth_city_region',
               'birth_city_population',
               'birth_city_latitude',
               'birth_city_longitude',
               'birth_city_country',
               'birth_country_code',
               'birth_country_iso3code',
               'birth_country_continent_code',
               'birth_country_capital',
               'birth_country_currency',
               'birth_country_phone',
               'birth_country_region',
               'birth_country_population',
               'birth_country_area',
               'birth_country_pop_density',
               'birth_country_coastline',
               'birth_country_net_migration',
               'birth_country_infant_mortality',
               'birth_country_gdp',
               'birth_country_literacy',
               'birth_country_phones',
               'birth_country_arable',
               'birth_country_crops',
               'birth_country_other',
               'birth_country_climate',
               'birth_country_birthrate',
               'birth_country_deathrate',
               'birth_country_agriculture',
               'birth_country_industry',
               'birth_country_service',
               
               'death_date',
               'death_city',
               'death_city_accentcity',
               'death_city_region',
               'death_city_population',
               'death_city_latitude',
               'death_city_longitude',
               'death_city_country',
               'death_country_code',
               'death_country_iso3code',
               'death_country_continent_code',
               'death_country_capital',
               'death_country_currency',
               'death_country_phone',
               'death_country_region',
               'death_country_population',
               'death_country_area',
               'death_country_pop_density',
               'death_country_coastline',
               'death_country_net_migration',
               'death_country_infant_mortality',
               'death_country_gdp',
               'death_country_literacy',
               'death_country_phones',
               'death_country_arable',
               'death_country_crops',
               'death_country_other',
               'death_country_climate',
               'death_country_birthrate',
               'death_country_deathrate',
               'death_country_agriculture',
               'death_country_industry',
               'death_country_service') \
        .write.format("orc") \
        .mode('overwrite') \
        .option("compression","zlib") \
        .saveAsTable("andreynovikov_snowflake.dataset")